In [1]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

from ptplot import plotting, utilities

%reload_ext autoreload
%autoreload 2

In [2]:
data_directory = "../data/bigdatabowl_2021/"
play_descriptions = pd.read_csv(data_directory + "plays.csv")
player_tracking_data = dd.read_csv(data_directory + "week*.csv", parse_dates=["time"])

We'll pick a single play to look at, just for demonstration purposes. I've somewhat-randomly chosen a [long
Aaron Rodgers-Randall Cobb touchdown pass from Week 1 of 2018](https://www.youtube.com/watch?v=I40ILjNdwZM&feature=emb_err_woyt).

In [3]:
interesting_play = player_tracking_data[
    (player_tracking_data["gameId"] == 2018090912) &
    (player_tracking_data["playId"] == 3564)
].compute()
interesting_play["team_abbreviation"] = np.where(interesting_play["team"] == "home", "GB", "CHI")

In [4]:
interesting_play.loc[interesting_play["displayName"] == "Football", "team_abbreviation"] = np.nan

interesting_play.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,team_abbreviation
315754,2018-09-10 03:11:31.599000+00:00,33.82,23.05,0.00,0.00,0.00,79.98,244.08,None,497236.0,Jimmy Graham,80.0,TE,1,home,2018090912,3564,right,CROSS,GB
315755,2018-09-10 03:11:31.599000+00:00,35.71,44.29,0.27,0.08,0.03,265.03,235.01,None,2495108.0,Prince Amukamara,20.0,CB,1,away,2018090912,3564,right,NaN,CHI
315756,2018-09-10 03:11:31.599000+00:00,32.78,19.21,0.00,0.00,0.00,49.33,182.31,None,2495448.0,Randall Cobb,18.0,WR,1,home,2018090912,3564,right,IN,GB
315757,2018-09-10 03:11:31.599000+00:00,29.66,29.79,0.00,0.00,0.00,104.44,92.47,None,2506363.0,Aaron Rodgers,12.0,QB,1,home,2018090912,3564,right,NaN,GB
315758,2018-09-10 03:11:31.599000+00:00,36.88,32.58,2.20,0.78,0.22,223.23,274.52,None,2532961.0,Danny Trevathan,59.0,ILB,1,away,2018090912,3564,right,NaN,CHI


In [10]:
fig = plotting.plot_positions(
    interesting_play[interesting_play["event"] == "pass_forward"], "x", "y",
    uniform_number="jerseyNumber",
    home_away_identifier=lambda data: data["team"] == "home",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "Football").values,
    team_abbreviations="team_abbreviation"
)
fig.show()

In [34]:
fig = plotting.plot_tracks(
    interesting_play, "x", "y", "displayName",
    home_away_identifier=lambda data: data["team"] == "home",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "Football").values,
    team_abbreviations="team_abbreviation"
)
fig.show()

In [6]:
fig = plotting.animate_play(
    interesting_play, "x", "y", "frameId",
    uniform_number="jerseyNumber",
    home_away_identifier=lambda data: data["team"] == "home",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "Football").values,
    slider_label_generator=utilities.generate_time_elapsed_labels(interesting_play["time"].min(), "time"),
    team_abbreviations="team_abbreviation",
    events_of_interest="event"
)
fig.show()

In [7]:
post_script = """
console.log("{plot_id}");
var plot_div = document.getElementById("{plot_id}");
var main_svg = plot_div.querySelector(".main-svg");
main_svg.className += "wooooo";
"""
fig.write_html(
    "test.html", auto_play=False, include_plotlyjs="cdn", full_html=False, config={'displayModeBar': False},
    #post_script = post_script
)

In [9]:
fig = plotting.plot_frame(
    interesting_play[interesting_play["event"] == "pass_forward"], "y", "x",
    uniform_number="jerseyNumber",
    home_away_identifier=lambda data: data["team"] == "home",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "Football").values,
    team_abbreviations="team_abbreviation",
    fig="nfl_vertical"
)
fig.update_layout(width=400, height=700)
fig.show()